In [35]:
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
import math
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly.graph_objs import *
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql import Row
import numpy as np

In [2]:
path = "/Users/alvira/Desktop/distributedData/project/"
input_file_name = path+'msan-697/review_u200.json'

In [3]:
reviews = sqlContext.read.json(input_file_name)

In [4]:
reviews = reviews.select(["user_id","business_id","stars"])

In [5]:
categoryIndexer = StringIndexer(inputCol="user_id", outputCol="label_user_id")
pipeline = Pipeline(stages=[categoryIndexer])
model = pipeline.fit(reviews)
reviews = model.transform(reviews)
categoryIndexer = StringIndexer(inputCol="business_id", outputCol="label_business_id")
pipeline = Pipeline(stages=[categoryIndexer])
model = pipeline.fit(reviews)
reviews = model.transform(reviews)

In [6]:
reviews.show()

+--------------------+--------------------+-----+-------------+-----------------+
|             user_id|         business_id|stars|label_user_id|label_business_id|
+--------------------+--------------------+-----+-------------+-----------------+
|P2E31XC7SP4v_3hXC...|5GcHJvxiTrIt23gB0...|    2|        363.0|           5411.0|
|Wu0yySWcHQ5tZ_59H...|5GcHJvxiTrIt23gB0...|    2|         50.0|           5411.0|
|0BBUmH7Krcax1RZgb...|5GcHJvxiTrIt23gB0...|    4|         27.0|           5411.0|
|CxDOIDnH8gp9KXzpB...|5GcHJvxiTrIt23gB0...|    3|          0.0|           5411.0|
|SxMBY64n31UHPjxzI...|5GcHJvxiTrIt23gB0...|    1|        271.0|           5411.0|
|YRcaNlwQ6XXPFDXWt...|FCqMWanlmta0fIGXW...|    3|        200.0|           4484.0|
|1fNQRju9gmoCEvbPQ...|FCqMWanlmta0fIGXW...|    3|        111.0|           4484.0|
|J3ucveGKKJDvtuCNn...|FCqMWanlmta0fIGXW...|    1|        202.0|           4484.0|
|WHmabbfgdSW-JrOU9...|FCqMWanlmta0fIGXW...|    3|        140.0|           4484.0|
|FREeRQtjdJU83AF

In [7]:
review_rdd = reviews.select(["label_user_id","label_business_id","stars"]).rdd

In [8]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [25]:
training, test = review_rdd.randomSplit([0.8, 0.2])

In [26]:
rank = 10
numIterations = 10
model = ALS.train(training, rank, numIterations)

In [27]:
# Evaluate the model on training data
testdata = test.map(lambda p: (p[0], p[1]))

In [28]:
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))

In [41]:
ratesAndPreds = test.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)

In [42]:
def double2int(x):
    val = np.round(x)
    if val < 0: val = 0
    return val

In [43]:
ratesAndPreds  = ratesAndPreds.map(lambda x: (x[0], (x[1][0], double2int(x[1][1]))) )

In [45]:
MSE = ratesAndPreds.map(lambda x: (x[1][0] - x[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

# Save and load model
# model.save(sc, "target/tmp/myCollaborativeFilter")
# sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")

Mean Squared Error = 3.36554570091


In [44]:
ratesAndPreds.collect()

[((12.0, 354.0), (3, 5.0)),
 ((273.0, 2885.0), (4, 4.0)),
 ((241.0, 317.0), (5, 4.0)),
 ((51.0, 10079.0), (5, 0.0)),
 ((181.0, 145.0), (3, 3.0)),
 ((102.0, 988.0), (4, 3.0)),
 ((176.0, 726.0), (4, 3.0)),
 ((51.0, 9047.0), (1, 0.0)),
 ((226.0, 5688.0), (4, 2.0)),
 ((234.0, 7504.0), (4, 2.0)),
 ((127.0, 995.0), (5, 2.0)),
 ((14.0, 4212.0), (4, 2.0)),
 ((152.0, 1942.0), (4, 3.0)),
 ((148.0, 2010.0), (3, 4.0)),
 ((215.0, 4627.0), (3, 2.0)),
 ((147.0, 559.0), (4, 4.0)),
 ((72.0, 5686.0), (4, 2.0)),
 ((17.0, 405.0), (3, 2.0)),
 ((2.0, 3776.0), (3, 2.0)),
 ((324.0, 10.0), (4, 4.0)),
 ((166.0, 12028.0), (3, 0.0)),
 ((289.0, 1693.0), (3, 4.0)),
 ((60.0, 3506.0), (3, 2.0)),
 ((274.0, 2432.0), (5, 4.0)),
 ((176.0, 70.0), (4, 4.0)),
 ((181.0, 377.0), (5, 1.0)),
 ((30.0, 6524.0), (4, 3.0)),
 ((15.0, 507.0), (4, 5.0)),
 ((235.0, 2863.0), (3, 5.0)),
 ((152.0, 982.0), (1, 1.0)),
 ((93.0, 6321.0), (3, 1.0)),
 ((9.0, 285.0), (4, 4.0)),
 ((201.0, 5853.0), (4, 1.0)),
 ((266.0, 3768.0), (4, 1.0)),
 ((4.0, 

In [18]:
testdata.count()

17377